In [4]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')

In [5]:
#just first 100 rows of train
train = train[:10]

In [13]:
train

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4
5,5,0,0,0.00,0,1.000635,13552875.92,NaN,NaN,0.999779,1962.72,1.000635,5647.65,1.0,6.779432,0,0_0_5
6,6,0,0,969969.40,1,1.000115,3647503.98,NaN,NaN,0.999506,6663.16,1.000283,3810.48,1.0,-2.499819,0,0_0_6
7,7,0,0,9412959.10,1,0.999818,21261245.87,NaN,NaN,0.999741,5139.20,1.000130,2570.60,1.0,-1.959801,0,0_0_7
8,8,0,0,2394875.85,1,0.999916,9473209.08,NaN,NaN,0.999022,52011.60,1.000041,2169.36,1.0,-5.970001,0,0_0_8
9,9,0,0,3039700.65,-1,1.000969,6248958.45,NaN,NaN,0.999354,6191.00,1.000646,6199.00,1.0,7.970333,0,0_0_9


In [6]:
#do train test split
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Assuming df is your DataFrame and it includes a 'target' column
X = train.drop('target', axis=1)  # feature columns
y = train['target']  # target column


#can u remove the NaNs
X = X.fillna(0)
y = y.fillna(0)
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [15]:
#remove nans in X_train and y_train
X_train = X_train.fillna(0)
y_train = y_train.fillna(0)


In [20]:
#xtrain and ytrain

import gym
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Assuming 'train' is your original dataset without the 'target' column
# You may need to preprocess and normalize your data appropriately

# Convert data types explicitly to float32
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)

# Define the RL environment
class StockTradingEnvironment(gym.Env):
    def __init__(self, data):
        super(StockTradingEnvironment, self).__init__()
        self.data = data
        self.current_step = 0
        self.max_steps = len(data) - 1

        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=-400, high=400, shape=(len(data.columns),), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        return self._next_observation()

    def step(self, action):
        self.current_step += 1
        observation = self._next_observation()
        reward = self._take_action(action)
        done = self.current_step == self.max_steps - 1

        return observation, reward, done, {}

    def _next_observation(self):
        return np.array(self.data.iloc[self.current_step])

    def _take_action(self, action):
        current_price = self.data.iloc[self.current_step]['reference_price']
        next_price = self.data.iloc[self.current_step + 1]['reference_price']

        # Scale the action to represent the change in basis points
        action = action * (next_price - current_price)

        reward = action  # Reward is the change in basis points
        return reward

# Create an instance of the environment
env = StockTradingEnvironment(X_train)

# Define the RL model
model = Sequential([
    Dense(64, activation='relu', input_shape=(len(X_train.columns),)),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')
])

model.compile(optimizer='adam', loss='mse')

# Training loop
EPISODES = 50  # You can adjust the number of episodes
MAX_STEPS = len(X_train) - 1
GAMMA = 0.95  # Discount factor

for episode in range(EPISODES):
    state = env.reset()
    state = np.reshape(state, [1, len(X_train.columns)])

    for step in range(MAX_STEPS):
        action = model.predict(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, len(X_train.columns)])

        target = reward + GAMMA * model.predict(next_state)
        
        model.fit(state, target, epochs=1, verbose=0)

        state = next_state

        if done:
            break

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')




1/1 [==============================] - 0s 13ms/step
Mean Absolute Error: 66178.7734375


In [21]:
print (y_pred)

[[-94447.41]
 [-37921.63]]
